---
format:
    html: 
        html-table-processing: none
---

In [ ]:
#| include: false

import polars as pl
import polars.selectors as cs 
from great_tables import GT, loc, style, html
from pyprojroot.here import here

here()

In [ ]:
#| include: false

df = (
    pl.read_csv(here("publications/citations.csv"))
    .select(["Year", "Title", "Publication", "Type","Authors","URL"])
    .rename({"Publication": "Venue"})
    .sort(by="Year", descending=True)
    .with_columns(
        pl.col("Venue")
        .str.to_titlecase()
        .str.replace("Of", "of")
        .str.replace("And", "and")
    )
    .with_columns(
        pl.when(pl.col("Title") == "A Biocultural Approach to Understanding the Impacts of Psychosocial Stress")
        .then(pl.lit("University of Florida"))
        .when(pl.col("Title") == "Discrimination, cultural consonance, and cell-mediated immunity among college students at the University of Alabama")
        .then(pl.lit("University of Alabama"))
        .otherwise(pl.col("Venue"))
        .alias("Venue")
    )
    .with_columns(
        pl.concat_str(
            [pl.lit("["), pl.col("Type"), pl.lit("]("), pl.col("URL"), pl.lit(")")], separator=""
        ).alias("Type")
    )
    .drop("URL")
)

print(df)

Add links to open access articles, or links to the open access PDFs, masters thesis, uf doctoral dissertations, etc.


In [ ]:
#| echo: false
#| output: true

gt1 = (
    GT(df)
    .tab_header(
        title="Publications"
    )
    .tab_style(
        style=style.text(style='italic'),
        locations=loc.body(columns="Venue"))
    # .tab_style(
    #     style=style.text(align="center"),
    #     locations=loc.column_labels()
    # )
    .opt_horizontal_padding(scale=3)
    .opt_vertical_padding(scale=0.5)
    .fmt_markdown(columns=["Type"])
)

gt1